In [1]:
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
pd.options.mode.chained_assignment = None

In [2]:
test = pd.read_csv('../data/test.csv', usecols=['item_id'])

In [3]:
periods = pd.read_csv('periods.csv')

In [4]:
tabular = pd.read_csv('tabular.csv')

In [5]:
nlp = pd.read_csv('nlp.csv')

In [6]:
test['periods_weight'] = 1.0
test['tabular_weight'] = 1.0
test['nlp_weight'] = 1.0

In [7]:
len(test)

508438

In [8]:
test.isna().sum() / len(test)

item_id           0.0
periods_weight    0.0
tabular_weight    0.0
nlp_weight        0.0
dtype: float64

In [9]:
test = test.merge(periods, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'periods'})
test['periods_weight'][null_idx] = 0.0
test['periods'][null_idx] = 0.0

In [10]:
test = test.merge(tabular, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'tabular'})
test['tabular_weight'][null_idx] = 0.0
test['tabular'][null_idx] = 0.0

In [11]:
test = test.merge(nlp, how='left', on='item_id')
null_idx = test['deal_probability'].isna()
test = test.rename(columns={'deal_probability': 'nlp'})
test['nlp_weight'][null_idx] = 0.0
test['nlp'][null_idx] = 0.0

In [45]:
test['deal_probability'] = test['periods']*test['periods_weight'] + \
                            test['tabular']*test['tabular_weight'] + \
                            test['nlp']*test['nlp_weight']
test['deal_probability'] /= ( test['periods_weight'] + test['tabular_weight'] + test['nlp_weight'])

In [55]:
submission = test[['item_id', 'deal_probability']].set_index('item_id')

In [56]:
### methods for transformation to [0, 1] range:
### 1) just cut out the negative part:
submission[submission < 0] = 0.0

### 2) MinMaxScaler to [0,1] range
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# submission['deal_probability'] = scaler.fit_transform(submission)

submission.min()

deal_probability    0.0
dtype: float64

In [57]:
submission.to_csv('submission.csv')

In [25]:
len(submission)

508438

In [36]:
submission

,deal_probability
item_id,
6544e41a8817,0.097277
65b9484d670f,0.276881
8bab230b2ecd,0.167920
8e348601fefc,0.286058
8bd2fe400b89,0.207867
c63dbd6c657f,0.163843
6d1a410df86e,0.136783
e8d3e7922b80,0.076400
2bc1ab208462,0.119701
